In [45]:
from mpa.models import Mpa, mpa_post_save
from mpa.views import mpas_all_nogeom
from wdpa.models import Wdpa2014Polygon, Wdpa2014Point, Wdpa2018Poly, Wdpa2018Point
from django.db.models import Q, F, Func
from importlib import reload

In [29]:
zeroes = mpas_all_nogeom.filter(marine=0, wdpa_id__gt=0)
# Func(F('wdpaid'), function='INTEGER', template='(%(expressions)s::%(function)s)::text')

In [30]:
zeroes.count()

1993

In [31]:
zids = zeroes.values_list('wdpa_id')
zpids = zeroes.exclude(wdpa_pid__isnull=True).values_list('wdpa_pid')
len(list(set(zpids)))
len(list(set(zids)))

1941

In [88]:
zeroes = zeroes.exclude(verification_state='Rejected as MPA')
fixed = []
count = 0
total = zeroes.count()
for z in zeroes.exclude(verification_state='Rejected as MPA'):
    count += 1
    # print(z.name, z.designation, z.wdpa_id, z.wdpa_pid)
    pid = None
    if z.wdpa_pid != '' and z.wdpa_pid is not None:
        # valid wdpa_pid
        if z.wdpa_pid == str(z.wdpa_id):
            # equal id and pid
            pid = z.wdpa_pid
        else:
            # zone: unequal id and pid
            pid = z.wdpa_pid
    else:
        # wdpa_pid is None or blank
        pid = z.wdpa_id
    w2018 = None
    w2014 = None
    try:
        w2018 = Wdpa2018Poly.objects.defer(*Wdpa2014Polygon.get_geom_fields()).get(wdpa_pid=pid)
    except:
        w2018 = Wdpa2018Point.objects.get(wdpa_pid=pid)
    try:
        w2014 = Wdpa2014Polygon.objects.defer(*Wdpa2014Polygon.get_geom_fields()).get(wdpa_pid=pid)
    except:
        try:
            w2014 = Wdpa2014Point.objects.get(wdpa_pid=pid)
        except:
            try:
                w2014 = Wdpa2014Polygon.objects.defer(*Wdpa2014Polygon.get_geom_fields()).get(wdpaid=z.wdpa_id)
            except:
                try:
                    w2014 = Wdpa2014Point.objects.get(wdpaid=z.wdpa_id)
                except:
                    pass
    # w2014 is optional, it might not exist
    if int(w2018.marine) > 0:
        print('2018 non-zero:', z.mpa_id, w2018.wdpa_pid, w2018.marine)
    if w2014 and w2014.marine != w2018.marine:
        print('%s/%s' % (count, total))
        print('2014/18 mismatch. 2018:', w2018.marine, '2014:', w2014.marine)
        if z.verification_state not in ('Internally Verified', 'Externally Verified'):
            # Keep original MPAtlas is_mpa value if verified,
            # otherwise set to is_mpa=False
            fixed.append(z.mpa_id)
            z.is_mpa = False;
            z.notes = z.notes + '\nSetting marine=0 and is_mpa=False because site is unverified.'
            z.save()
    if not w2014:
        print('No 2014 record:', z.mpa_id, w2018.wdpa_pid)
    

1/1717
2014/18 mismatch. 2018: 0 2014: 1
2/1717
2014/18 mismatch. 2018: 0 2014: 1
3/1717
2014/18 mismatch. 2018: 0 2014: 1
4/1717
2014/18 mismatch. 2018: 0 2014: 1
5/1717
2014/18 mismatch. 2018: 0 2014: 1
6/1717
2014/18 mismatch. 2018: 0 2014: 1
7/1717
2014/18 mismatch. 2018: 0 2014: 1
9/1717
2014/18 mismatch. 2018: 0 2014: 1
10/1717
2014/18 mismatch. 2018: 0 2014: 1
11/1717
2014/18 mismatch. 2018: 0 2014: 1
12/1717
2014/18 mismatch. 2018: 0 2014: 1
13/1717
2014/18 mismatch. 2018: 0 2014: 1
14/1717
2014/18 mismatch. 2018: 0 2014: 1
15/1717
2014/18 mismatch. 2018: 0 2014: 1
16/1717
2014/18 mismatch. 2018: 0 2014: 1
17/1717
2014/18 mismatch. 2018: 0 2014: 1
19/1717
2014/18 mismatch. 2018: 0 2014: 1
20/1717
2014/18 mismatch. 2018: 0 2014: 1
22/1717
2014/18 mismatch. 2018: 0 2014: 1
23/1717
2014/18 mismatch. 2018: 0 2014: 1
24/1717
2014/18 mismatch. 2018: 0 2014: 1
27/1717
2014/18 mismatch. 2018: 0 2014: 1
28/1717
2014/18 mismatch. 2018: 0 2014: 1
29/1717
2014/18 mismatch. 2018: 0 2014: 1


219/1717
2014/18 mismatch. 2018: 0 2014: 1
221/1717
2014/18 mismatch. 2018: 0 2014: 1
222/1717
2014/18 mismatch. 2018: 0 2014: 1
223/1717
2014/18 mismatch. 2018: 0 2014: 1
224/1717
2014/18 mismatch. 2018: 0 2014: 1
225/1717
2014/18 mismatch. 2018: 0 2014: 1
226/1717
2014/18 mismatch. 2018: 0 2014: 1
228/1717
2014/18 mismatch. 2018: 0 2014: 1
229/1717
2014/18 mismatch. 2018: 0 2014: 1
231/1717
2014/18 mismatch. 2018: 0 2014: 1
232/1717
2014/18 mismatch. 2018: 0 2014: 1
233/1717
2014/18 mismatch. 2018: 0 2014: 1
234/1717
2014/18 mismatch. 2018: 0 2014: 1
235/1717
2014/18 mismatch. 2018: 0 2014: 1
236/1717
2014/18 mismatch. 2018: 0 2014: 1
237/1717
2014/18 mismatch. 2018: 0 2014: 1
238/1717
2014/18 mismatch. 2018: 0 2014: 1
239/1717
2014/18 mismatch. 2018: 0 2014: 1
240/1717
2014/18 mismatch. 2018: 0 2014: 1
241/1717
2014/18 mismatch. 2018: 0 2014: 1
242/1717
2014/18 mismatch. 2018: 0 2014: 1
243/1717
2014/18 mismatch. 2018: 0 2014: 1
244/1717
2014/18 mismatch. 2018: 0 2014: 1
245/1717
20

440/1717
2014/18 mismatch. 2018: 0 2014: 1
441/1717
2014/18 mismatch. 2018: 0 2014: 1
442/1717
2014/18 mismatch. 2018: 0 2014: 1
443/1717
2014/18 mismatch. 2018: 0 2014: 1
444/1717
2014/18 mismatch. 2018: 0 2014: 1
445/1717
2014/18 mismatch. 2018: 0 2014: 1
446/1717
2014/18 mismatch. 2018: 0 2014: 1
447/1717
2014/18 mismatch. 2018: 0 2014: 1
448/1717
2014/18 mismatch. 2018: 0 2014: 1
449/1717
2014/18 mismatch. 2018: 0 2014: 1
450/1717
2014/18 mismatch. 2018: 0 2014: 1
451/1717
2014/18 mismatch. 2018: 0 2014: 1
452/1717
2014/18 mismatch. 2018: 0 2014: 1
453/1717
2014/18 mismatch. 2018: 0 2014: 1
454/1717
2014/18 mismatch. 2018: 0 2014: 1
455/1717
2014/18 mismatch. 2018: 0 2014: 1
457/1717
2014/18 mismatch. 2018: 0 2014: 1
458/1717
2014/18 mismatch. 2018: 0 2014: 1
459/1717
2014/18 mismatch. 2018: 0 2014: 1
460/1717
2014/18 mismatch. 2018: 0 2014: 1
461/1717
2014/18 mismatch. 2018: 0 2014: 1
462/1717
2014/18 mismatch. 2018: 0 2014: 1
463/1717
2014/18 mismatch. 2018: 0 2014: 1
464/1717
20

651/1717
2014/18 mismatch. 2018: 0 2014: 1
652/1717
2014/18 mismatch. 2018: 0 2014: 1
653/1717
2014/18 mismatch. 2018: 0 2014: 1
654/1717
2014/18 mismatch. 2018: 0 2014: 1
655/1717
2014/18 mismatch. 2018: 0 2014: 1
657/1717
2014/18 mismatch. 2018: 0 2014: 1
659/1717
2014/18 mismatch. 2018: 0 2014: 1
660/1717
2014/18 mismatch. 2018: 0 2014: 1
661/1717
2014/18 mismatch. 2018: 0 2014: 1
662/1717
2014/18 mismatch. 2018: 0 2014: 1
663/1717
2014/18 mismatch. 2018: 0 2014: 1
667/1717
2014/18 mismatch. 2018: 0 2014: 1
668/1717
2014/18 mismatch. 2018: 0 2014: 1
669/1717
2014/18 mismatch. 2018: 0 2014: 1
670/1717
2014/18 mismatch. 2018: 0 2014: 1
671/1717
2014/18 mismatch. 2018: 0 2014: 1
672/1717
2014/18 mismatch. 2018: 0 2014: 1
673/1717
2014/18 mismatch. 2018: 0 2014: 1
674/1717
2014/18 mismatch. 2018: 0 2014: 1
676/1717
2014/18 mismatch. 2018: 0 2014: 1
677/1717
2014/18 mismatch. 2018: 0 2014: 1
678/1717
2014/18 mismatch. 2018: 0 2014: 1
679/1717
2014/18 mismatch. 2018: 0 2014: 1
680/1717
20

871/1717
2014/18 mismatch. 2018: 0 2014: 1
874/1717
2014/18 mismatch. 2018: 0 2014: 1
875/1717
2014/18 mismatch. 2018: 0 2014: 1
876/1717
2014/18 mismatch. 2018: 0 2014: 1
877/1717
2014/18 mismatch. 2018: 0 2014: 1
878/1717
2014/18 mismatch. 2018: 0 2014: 1
879/1717
2014/18 mismatch. 2018: 0 2014: 1
880/1717
2014/18 mismatch. 2018: 0 2014: 1
881/1717
2014/18 mismatch. 2018: 0 2014: 1
882/1717
2014/18 mismatch. 2018: 0 2014: 1
883/1717
2014/18 mismatch. 2018: 0 2014: 1
884/1717
2014/18 mismatch. 2018: 0 2014: 1
885/1717
2014/18 mismatch. 2018: 0 2014: 1
886/1717
2014/18 mismatch. 2018: 0 2014: 1
888/1717
2014/18 mismatch. 2018: 0 2014: 1
889/1717
2014/18 mismatch. 2018: 0 2014: 1
890/1717
2014/18 mismatch. 2018: 0 2014: 1
891/1717
2014/18 mismatch. 2018: 0 2014: 1
892/1717
2014/18 mismatch. 2018: 0 2014: 1
893/1717
2014/18 mismatch. 2018: 0 2014: 1
894/1717
2014/18 mismatch. 2018: 0 2014: 1
895/1717
2014/18 mismatch. 2018: 0 2014: 1
896/1717
2014/18 mismatch. 2018: 0 2014: 1
897/1717
20

1077/1717
2014/18 mismatch. 2018: 0 2014: 1
1078/1717
2014/18 mismatch. 2018: 0 2014: 1
1079/1717
2014/18 mismatch. 2018: 0 2014: 1
1080/1717
2014/18 mismatch. 2018: 0 2014: 1
1081/1717
2014/18 mismatch. 2018: 0 2014: 1
1083/1717
2014/18 mismatch. 2018: 0 2014: 1
1084/1717
2014/18 mismatch. 2018: 0 2014: 1
1085/1717
2014/18 mismatch. 2018: 0 2014: 1
1086/1717
2014/18 mismatch. 2018: 0 2014: 1
1087/1717
2014/18 mismatch. 2018: 0 2014: 1
1088/1717
2014/18 mismatch. 2018: 0 2014: 1
1089/1717
2014/18 mismatch. 2018: 0 2014: 1
1090/1717
2014/18 mismatch. 2018: 0 2014: 1
1091/1717
2014/18 mismatch. 2018: 0 2014: 1
1092/1717
2014/18 mismatch. 2018: 0 2014: 1
1093/1717
2014/18 mismatch. 2018: 0 2014: 1
1094/1717
2014/18 mismatch. 2018: 0 2014: 1
1095/1717
2014/18 mismatch. 2018: 0 2014: 1
1096/1717
2014/18 mismatch. 2018: 0 2014: 1
1097/1717
2014/18 mismatch. 2018: 0 2014: 1
1098/1717
2014/18 mismatch. 2018: 0 2014: 1
1100/1717
2014/18 mismatch. 2018: 0 2014: 1
1101/1717
2014/18 mismatch. 2018

1310/1717
2014/18 mismatch. 2018: 0 2014: 1
1311/1717
2014/18 mismatch. 2018: 0 2014: 1
1312/1717
2014/18 mismatch. 2018: 0 2014: 1
1313/1717
2014/18 mismatch. 2018: 0 2014: 1
1314/1717
2014/18 mismatch. 2018: 0 2014: 1
1315/1717
2014/18 mismatch. 2018: 0 2014: 1
1316/1717
2014/18 mismatch. 2018: 0 2014: 1
1317/1717
2014/18 mismatch. 2018: 0 2014: 1
1318/1717
2014/18 mismatch. 2018: 0 2014: 1
1319/1717
2014/18 mismatch. 2018: 0 2014: 1
1320/1717
2014/18 mismatch. 2018: 0 2014: 1
1321/1717
2014/18 mismatch. 2018: 0 2014: 1
1322/1717
2014/18 mismatch. 2018: 0 2014: 1
1323/1717
2014/18 mismatch. 2018: 0 2014: 1
1324/1717
2014/18 mismatch. 2018: 0 2014: 1
1325/1717
2014/18 mismatch. 2018: 0 2014: 1
1326/1717
2014/18 mismatch. 2018: 0 2014: 1
1327/1717
2014/18 mismatch. 2018: 0 2014: 1
1328/1717
2014/18 mismatch. 2018: 0 2014: 1
1329/1717
2014/18 mismatch. 2018: 0 2014: 1
1330/1717
2014/18 mismatch. 2018: 0 2014: 1
1331/1717
2014/18 mismatch. 2018: 0 2014: 1
1332/1717
2014/18 mismatch. 2018

1517/1717
2014/18 mismatch. 2018: 0 2014: 1
1518/1717
2014/18 mismatch. 2018: 0 2014: 1
1519/1717
2014/18 mismatch. 2018: 0 2014: 1
1520/1717
2014/18 mismatch. 2018: 0 2014: 1
1521/1717
2014/18 mismatch. 2018: 0 2014: 1
1522/1717
2014/18 mismatch. 2018: 0 2014: 1
1524/1717
2014/18 mismatch. 2018: 0 2014: 1
1526/1717
2014/18 mismatch. 2018: 0 2014: 1
1528/1717
2014/18 mismatch. 2018: 0 2014: 1
1530/1717
2014/18 mismatch. 2018: 0 2014: 1
1531/1717
2014/18 mismatch. 2018: 0 2014: 1
1533/1717
2014/18 mismatch. 2018: 0 2014: 1
1534/1717
2014/18 mismatch. 2018: 0 2014: 1
1535/1717
2014/18 mismatch. 2018: 0 2014: 1
1537/1717
2014/18 mismatch. 2018: 0 2014: 1
1538/1717
2014/18 mismatch. 2018: 0 2014: 1
1539/1717
2014/18 mismatch. 2018: 0 2014: 1
1540/1717
2014/18 mismatch. 2018: 0 2014: 1
1541/1717
2014/18 mismatch. 2018: 0 2014: 1
1543/1717
2014/18 mismatch. 2018: 0 2014: 1
1544/1717
2014/18 mismatch. 2018: 0 2014: 1
1545/1717
2014/18 mismatch. 2018: 0 2014: 1
1546/1717
2014/18 mismatch. 2018

In [89]:
fixed

[13158,
 11532,
 14027,
 1728,
 10846,
 10211,
 13344,
 11597,
 11598,
 11847,
 1792,
 1860,
 13490,
 11345,
 13623,
 13462,
 13633,
 1794,
 1797,
 1798,
 6576,
 6578,
 6581,
 1846,
 7477,
 13529,
 13534,
 13538,
 6944,
 10217,
 13542,
 12587,
 13665,
 1691,
 1718,
 1722,
 1724,
 1725,
 1727,
 13548,
 13552,
 10223,
 10593,
 1793,
 10210,
 13556,
 7349,
 2108,
 9939,
 10209,
 5873,
 4175,
 2066,
 2106,
 13103,
 1125,
 1127,
 1130,
 1405,
 1454,
 1483,
 1484,
 2267,
 13086,
 13087,
 2287,
 3284,
 11520,
 2897,
 13899,
 4176,
 11524,
 793,
 925,
 11531,
 10218,
 10635,
 794,
 9945,
 10638,
 2661,
 13589,
 13590,
 13594,
 13595,
 7700396,
 6274,
 13366,
 1191,
 7701199,
 1417,
 13612,
 13614,
 265,
 6940,
 6947,
 6951,
 6953,
 12645,
 6334,
 9913,
 12646,
 13636,
 13639,
 12465,
 13092,
 13642,
 13641,
 951,
 980,
 12488,
 9926,
 6314,
 1199,
 10884,
 9928,
 12294,
 13647,
 11593,
 1365,
 13650,
 13651,
 9943,
 13054,
 13655,
 799,
 12492,
 13659,
 13663,
 1916,
 12498,
 13669,
 12867,
 9

In [93]:
Mpa.objects.filter(mpa_id__in=fixed).update(verification_state='Rejected as MPA', verification_reason='Unverified site that WDPA has set as marine=0 as of October 2018')

1074

In [86]:
hc = mpas_all_nogeom.filter(sovereign__icontains='GBR', designation__icontains='National Nature Reserve')
for h in hc:
    print(h.name, h.designation)
    h.is_mpa=False
    h.verification_state='Rejected as MPA'
    h.save()


Rum National Nature Reserve
Caerlaverock National Nature Reserve
Lindisfarne National Nature Reserve
Bridgwater Bay National Nature Reserve
Dyfi National Nature Reserve
St Kilda National Nature Reserve
Noss National Nature Reserve
Newborough Warren and Ynys Llanddwyn National Nature Reserve
Morfa Harlech National Nature Reserve
Ribble Estuary National Nature Reserve
Scolt Head Island National Nature Reserve
Blakeney National Nature Reserve
Holkham National Nature Reserve
Hermaness National Nature Reserve
Rona and Sula Sgeir National Nature Reserve
St Abb's Head National Nature Reserve
Monach Isles National Nature Reserve
Dengie National Nature Reserve
Ainsdale Sand Dunes National Nature Reserve
Arne Reedbeds National Nature Reserve
Blackwater Estuary National Nature Reserve
Colne Estuary National Nature Reserve
Gibraltar Point National Nature Reserve
Hamford Water National Nature Reserve
Leigh National Nature Reserve
North Solent National Nature Reserve
Orfordness-havergate National Na